## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
dir_path = "/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Raw_Recordings"

In [4]:
x = np.load(dir_path + "/tr_x.npy", allow_pickle= True)
y = np.load(dir_path + "/tr_y.npy", allow_pickle= True)
z = np.load(dir_path + "/tr_z.npy", allow_pickle= True)

In [5]:
x.shape

(100,)

## Load csv and create two seperate dataframe:
We are going to use heart dataframe to differentiate healty people from the people with diaseaes. Data's of healty people will be stored in the healty dataset while diseased people will be in diseased dataset.

In [6]:
heart = pd.read_excel("/Users/ecem/Documents/GitHub/seismocardiogram/data/diseased-dataset/Summary_Pub_Deidentified.xlsx")
df = heart.copy().fillna(0)

I have dropped some columns because they didnt have much information. ????????

In [7]:
df.drop(["Mitral valve Peak velocity (m/sec)",
         "Mitral valve Mean gradient(mmHg)", 
         "Echo available",
        "Date of collection",
        "Date of echo"], 
        inplace = True, axis = 1)
df.drop(df.tail(1).index,inplace=True)

In [8]:
df.index[-1]

99

In [9]:
x[99]

... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:04:59.621094Z | 256.0 Hz, 76704 samples

## MS 

In [10]:
idx_MS = np.where((df['Moderate or greater MS'] == 1.0)  
                  &(df['Moderate or greater MR'] == 0.0)  
                  & (df['Moderate or greater AR'] == 0.0)
                  &(df['Moderate or greater AS'] == 0.0)  
                  & (df['moderate or greater TR'] == 0.0))


In [11]:
idx_MS

(array([ 8, 28, 62]),)

In [12]:
diseased_MS = df.loc[idx_MS]

In [13]:
idx_AR = np.where((df['Moderate or greater MS'] == 0.0)  
                  &(df['Moderate or greater MR'] == 0.0)  
                  & (df['Moderate or greater AR'] == 1.0)
                  &(df['Moderate or greater AS'] == 0.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [14]:
idx_AR

(array([17, 21, 26, 39, 68]),)

In [15]:
diseased_AR = df.loc[idx_AR]

In [16]:
idx_MR = np.where((df['Moderate or greater MS'] == 0.0)  
                  &(df['Moderate or greater MR'] == 1.0)  
                  & (df['Moderate or greater AR'] == 0.0)
                  &(df['Moderate or greater AS'] == 0.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [17]:
idx_MR

(array([ 0,  2, 12, 19, 27, 31, 32, 37, 45, 50, 54, 60, 65]),)

In [18]:
diseased_MR = df.loc[idx_MR]

In [19]:
idx_AS = np.where((df['Moderate or greater MS'] == 0.0)  
                  &(df['Moderate or greater MR'] == 0.0)  
                  & (df['Moderate or greater AR'] == 0.0)
                  &(df['Moderate or greater AS'] == 1.0)  
                  & (df['moderate or greater TR'] == 0.0))

In [20]:
idx_AS

(array([44, 71, 73, 76, 78, 80, 84, 85, 87, 88, 92, 95, 96, 97, 98, 99]),)

In [21]:
diseased_AS = df.loc[idx_AS]

In [22]:
diseased_AS

Patient ID     Start  Duration  Sampling rate(Hz)   Age Gender  \
44      CP-45  00:00:33  00:06:19         256.000000  70.0      M   
71      UP-02  00:00:00  00:06:10         256.000000  89.0      M   
73      UP-04  00:00:00  00:05:36         256.000000  84.0      M   
76      UP-07  00:00:00  00:06:03         256.000000  84.0      M   
78      UP-09  00:00:00  00:05:15         256.000000  89.0      M   
80      UP-11  00:00:00  00:05:06         256.000000  68.0      M   
84      UP-15  00:00:00  00:05:34         256.000000  84.0      F   
85      UP-16  00:00:00  00:05:44         256.000000  97.0      M   
87      UP-18  00:00:00  00:05:09         256.000000  90.0      F   
88      UP-19  00:00:00  00:05:17         256.000000  78.0      M   
92      UP-23  00:00:00  00:09:59         512.041667  84.0      F   
95      UP-26  00:00:00  00:05:08         512.166667  80.0      M   
96      UP-27  00:00:00  00:05:05         512.208333  82.0      M   
97      UP-28  00:00:00  00:05:02         512.250000  71.0      M   
98      UP-29  00:00:00  00:04:58         512.291667  80.0      M   
99      UP-30  00:00:00  00:04:59         512.333333  71.0      M   

    Height (cm)  Weight (kg)  History of MI  History of CABG  ...  TAPSE  \
44        170.0         80.0            0.0              0.0  ...   21.0   
71        170.2         82.0            1.0              1.0  ...    0.0   
73        152.4         65.0            0.0              1.0  ...    0.0   
76        162.5         79.0            0.0              1.0  ...    0.0   
78        182.8         90.7            0.0              1.0  ...    0.0   
80        177.8         79.0            0.0              1.0  ...    0.0   
84        175.0         76.0            0.0              1.0  ...    0.0   
85        157.0         77.0            0.0              1.0  ...    0.0   
87        152.4         92.0            0.0              1.0  ...    0.0   
88        170.1         78.0            0.0              1.0  ...    0.0   
92        139.7         70.3            0.0              0.0  ...    0.0   
95        175.3         85.7            0.0              0.0  ...    0.0   
96        180.0        118.0            0.0              0.0  ...    0.0   
97        175.0        117.0            0.0              0.0  ...    0.0   
98        168.9         65.8            0.0              0.0  ...    0.0   
99        177.8         81.6            0.0              0.0  ...    0.0   

    Aortic valve area (cm2)  Aortic valve area Mean gradient(mmHg)  \
44                     0.90                                   39.0   
71                     1.00                                   24.6   
73                     0.90                                   34.6   
76                     0.40                                   51.3   
78                     0.75                                   59.8   
80                     0.93                                   54.4   
84                     0.90                                   33.8   
85                     0.98                                   22.8   
87                     1.00                                   11.1   
88                     0.99                                   27.3   
92                     0.00                                   38.2   
95                     0.90                                   29.7   
96                     0.40                                   30.1   
97                     0.50                                   56.0   
98                     0.40                                   49.3   
99                     0.60                                   60.7   

    Aortic valve area Peak velocity (m/sec)  Mitral valve area (cm2)  \
44                                      4.3                      0.0   
71                                      3.2                      0.0   
73                                      4.2                      0.0   
76                                      4.7              

In [23]:
MS_x, MS_y, MS_z = [],[],[]
MR_x, MR_y, MR_z = [],[],[]

AS_x, AS_y, AS_z = [],[],[]
AR_x, AR_y, AR_z = [],[],[]


In [24]:
for i in range(len(diseased_MS['Patient ID'].index)):
    x_ = x[diseased_MS['Patient ID'].index[i]]
    y_ = y[diseased_MS['Patient ID'].index[i]]
    z_ = z[diseased_MS['Patient ID'].index[i]]
    
    MS_x.append(x_)
    MS_y.append(y_) 
    MS_z.append(z_) 
del x_, y_, z_

In [25]:
for i in range(len(diseased_AR['Patient ID'].index)):
    x_ = x[diseased_AR['Patient ID'].index[i]]
    y_ = y[diseased_AR['Patient ID'].index[i]]
    z_ = z[diseased_AR['Patient ID'].index[i]]
    
    AR_x.append(x_)
    AR_y.append(y_) 
    AR_z.append(z_) 
del x_, y_, z_

In [26]:
for i in range(len(diseased_MR['Patient ID'].index)):
    x_ = x[diseased_MR['Patient ID'].index[i]]
    y_ = y[diseased_MR['Patient ID'].index[i]]
    z_ = z[diseased_MR['Patient ID'].index[i]]
    
    MR_x.append(x_)
    MR_y.append(y_) 
    MR_z.append(z_) 
del x_, y_, z_

In [27]:
for i in range(len(diseased_AS['Patient ID'].index)):
    x_ = x[diseased_AS['Patient ID'].index[i]]
    y_ = y[diseased_AS['Patient ID'].index[i]]
    z_ = z[diseased_AS['Patient ID'].index[i]]
    
    AS_x.append(x_)
    AS_y.append(y_) 
    AS_z.append(z_) 
del x_, y_, z_

In [28]:
AS_x

[... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:18.679688Z | 256.0 Hz, 112303 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:06:10.062500Z | 256.0 Hz, 94737 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:36.863281Z | 256.0 Hz, 86238 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:06:03.011719Z | 256.0 Hz, 92932 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:15.605469Z | 256.0 Hz, 80796 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:06.742188Z | 256.0 Hz, 78527 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:34.824219Z | 256.0 Hz, 85716 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:44.816406Z | 256.0 Hz, 88274 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:09.382812Z | 256.0 Hz, 79203 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:17.289062Z | 256.0 Hz, 81227 samples,
 ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:09:59.757812Z | 256.0 Hz, 15

In [29]:
len(x)

100

In [30]:
#note to myself: do not change order
dataset1 = np.stack((MR_x, MR_y, MR_z), axis=-1)
dataset2 = np.stack((AR_x, AR_y, AR_z), axis=-1)
dataset3 = np.stack((AS_x, AS_y, AS_z), axis=-1)
dataset4 = np.stack((MS_x, MS_y, MS_z), axis=-1)

In [31]:
print(dataset1.shape)
print(dataset2.shape)
print(dataset3.shape)
print(dataset4.shape)


(13, 3)
(5, 3)
(16, 3)
(3, 3)


In [32]:
ms_w = np.load(dir_path + "/ms-without-detrend-taper.npy", allow_pickle= True)
mr_w = np.load(dir_path + "/mr-without-detrend-taper.npy", allow_pickle= True)
as_w = np.load(dir_path + "/as-without-detrend-taper.npy", allow_pickle= True)
ar_w = np.load(dir_path + "/ar-without-detrend-taper.npy", allow_pickle= True)

In [33]:
ms_ = np.load(dir_path + "/MS.npy", allow_pickle= True)
mr_ = np.load(dir_path + "/MR.npy", allow_pickle= True)
as_ = np.load(dir_path + "/AS.npy", allow_pickle= True)
ar_ = np.load(dir_path + "/AR.npy", allow_pickle= True)

In [34]:
mr_.shape

(13, 3)

In [35]:
for i in range(13):
    print("***************{}".format(i))
    print("with", mr_[i][0])
    print("without", mr_w[i][0])

***************0
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:07.640625Z | 256.0 Hz, 124837 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:07.640625Z | 256.0 Hz, 124837 samples
***************1
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:09:56.785156Z | 256.0 Hz, 152778 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:09:56.785156Z | 256.0 Hz, 152778 samples
***************2
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:56.488281Z | 256.0 Hz, 137342 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:56.488281Z | 256.0 Hz, 137342 samples
***************3
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:56.722656Z | 256.0 Hz, 122042 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:56.722656Z | 256.0 Hz, 122042 samples
***************4
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:38.730469Z | 256.0 Hz, 132796 samples
without ... | 1970-01-01T00:00:00.00000

In [36]:
for i in range(5):
    print("***************{}".format(i))
    print("with", ar_[i][0])
    print("without", ar_w[i][0])

***************0
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:10:21.855469Z | 256.0 Hz, 159196 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:10:21.855469Z | 256.0 Hz, 159196 samples
***************1
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:57.121094Z | 256.0 Hz, 122144 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:57.121094Z | 256.0 Hz, 122144 samples
***************2
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:59.531250Z | 256.0 Hz, 138121 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:59.531250Z | 256.0 Hz, 138121 samples
***************3
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:45.871094Z | 256.0 Hz, 134624 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:45.871094Z | 256.0 Hz, 134624 samples
***************4
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:55.304688Z | 256.0 Hz, 121679 samples
without ... | 1970-01-01T00:00:00.00000

In [37]:
for i in range(3):
    print("***************{}".format(i))
    print("with", ms_[i][0])
    print("without", ms_w[i][0])

***************0
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:56.488281Z | 256.0 Hz, 137342 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:08:56.488281Z | 256.0 Hz, 137342 samples
***************1
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:09:33.109375Z | 256.0 Hz, 146717 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:09:33.109375Z | 256.0 Hz, 146717 samples
***************2
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:31.902344Z | 256.0 Hz, 115688 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:31.902344Z | 256.0 Hz, 115688 samples


In [38]:
for i in range(16):
    print("***************{}".format(i))
    print("with", as_[i][0])
    print("without", as_w[i][0])

***************0
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:18.679688Z | 256.0 Hz, 112303 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:07:18.679688Z | 256.0 Hz, 112303 samples
***************1
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:06:10.062500Z | 256.0 Hz, 94737 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:06:10.062500Z | 256.0 Hz, 94737 samples
***************2
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:36.863281Z | 256.0 Hz, 86238 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:36.863281Z | 256.0 Hz, 86238 samples
***************3
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:06:03.011719Z | 256.0 Hz, 92932 samples
without ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:06:03.011719Z | 256.0 Hz, 92932 samples
***************4
with ... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:05:15.605469Z | 256.0 Hz, 80796 samples
without ... | 1970-01-01T00:00:00.000000Z - 19

In [39]:
np.save(dir_path + "/MR.npy", dataset1, allow_pickle= True)
np.save(dir_path + "/AR.npy", dataset2, allow_pickle= True)
np.save(dir_path + "/AS.npy", dataset3, allow_pickle= True)
np.save(dir_path + "/MS.npy", dataset4, allow_pickle= True)